In [243]:
from astropy.table import Table, join
import numpy as np

import time
from numpy.random import default_rng

import healpy as hp
import pandas as pd

import astropy.cosmology
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Table
from astropy.cosmology import Planck18

import sys
sys.path.insert(0,'../code')
%load_ext autoreload
%autoreload 2
import utils
import generate_random
import correlations
import masks
import maps

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameter setting

Saving

In [89]:
quants = {}

In [41]:
fn_quants = '../data/quantities.txt'
save = True

Names

In [3]:
name_catalog = '\emph{{Gaia}} Cosmological Quasar Catalog'
abbrv_catalog = 'GaiaQ'

name_gpurer = '\emph{{Gaia}} Purer Sample'
abbrv_gpurer = '\emph{{Gaia}} Purer'

Functions

In [4]:
def get_ndens_map(ras, decs, NSIDE=64):
    map_nqso, _ = maps.get_map(NSIDE, ras, decs, null_val=0)
    map_ndens = map_nqso/area_per_pixel
    map_ndens_masked = hp.ma(map_ndens)
    map_ndens_masked.mask = map_ndens==0
    return map_ndens_masked

Map settings

In [5]:
NSIDE = 64
area_per_pixel = hp.nside2pixarea(NSIDE, degrees=True)
print(f"Area per pixel: {area_per_pixel:.3f} deg")

Area per pixel: 0.839 deg


Other prelims

In [6]:
G_hi = 20.5
G_lo = 20.0

In [7]:
rng = np.random.default_rng(seed=42)

# Load datasets

### Gaia

In [8]:
fn_gall = '../data/gaia_candidates.fits.gz'
tab_gall = utils.load_table(fn_gall)
print(len(tab_gall))

6649162


In [9]:
fn_gpurer_ids = '../data/gaia_purer_sourceids.fits'
tab_gpurer_ids = utils.load_table(fn_gpurer_ids)
tab_gpurer = join(tab_gall, tab_gpurer_ids, keys='source_id', join_type='inner', metadata_conflicts='silent')
print(len(tab_gpurer))

1942825


In [10]:
fn_gsup = '../data/gaia_candidates_superset.fits'
tab_gsup = utils.load_table(fn_gsup)
print(len(tab_gsup))

1518782


In [11]:
fn_gclean = '../data/gaia_candidates_clean.fits'
tab_gclean = utils.load_table(fn_gclean)
print(len(tab_gclean))

1442077


In [225]:
fn_gcatlo = f'../data/gaiaQ_G{G_lo}.fits'
tab_gcatlo = utils.load_table(fn_gcatlo)
print(len(tab_gcatlo))

762347


In [226]:
fn_gcathi = f'../data/gaiaQ_G{G_hi}.fits'
tab_gcathi = utils.load_table(fn_gcathi)
print(len(tab_gcathi))

1309315


### SDSS

These are SDSS objects that have (any) Gaia source matches. There is unWISE data where available, but the below catalogs are not limited to unWISE matches.

In [14]:
tab_squasars = utils.load_table(f'../data/quasars_sdss_xgaia_xunwise_good_nodup.fits')
print(f"Number of SDSS quasars: {len(tab_squasars)}")
print(f"Number with unWISE info:", np.sum(np.isfinite(tab_squasars['mag_w1_vg']) & ~tab_squasars.mask['mag_w1_vg']))

Number of SDSS quasars: 379698
Number with unWISE info: 350070


In [15]:
tab_sstars = utils.load_table(f'../data/stars_sdss_xgaia_xunwise_good_nodup.fits')
print(f"Number of SDSS stars: {len(tab_sstars)}")
print(f"Number with unWISE info:", np.sum(np.isfinite(tab_sstars['mag_w1_vg']) & ~tab_sstars.mask['mag_w1_vg']))

Number of SDSS stars: 683221
Number with unWISE info: 482080


In [16]:
tab_sgals = utils.load_table(f'../data/galaxies_sdss_xgaia_xunwise_good_nodup.fits')
print(f"Number of SDSS galaxies: {len(tab_sgals)}")
print(f"Number with unWISE info:", np.sum(np.isfinite(tab_sgals['mag_w1_vg']) & ~tab_sgals.mask['mag_w1_vg']))

Number of SDSS galaxies: 717059
Number with unWISE info: 600897


Only objects in superset (gaia qso candidates with unWISE data and QSOC redshifts and G lim)

In [17]:
fn_labeled_sup = '../data/labeled_superset.fits'
tab_labeled_sup = utils.load_table(fn_labeled_sup)
print(f"Number of labeled Gaia quasar candidates for decontamination: {len(tab_labeled_sup)}")

Number of labeled Gaia quasar candidates for training/validation: 249714


Redshifts

In [53]:
redshift_estimator_name = 'kNN'
save_tag_model = f'_K27_std'
fn_spz_labeled = f'../data/redshift_estimates/redshifts_spz_labeled_{redshift_estimator_name}{save_tag_model}.fits'
tab_spz_labeled = utils.load_table(fn_spz_labeled)
print(f"N = {len(tab_spz_labeled)}")

N = 245308


# Quantities

## Numbers of objects

In [90]:
quants['N_gall'] = f"{len(tab_gall):,}"
print(quants['N_gall'])

6,649,162


In [91]:
i_gall_wqsoc = np.isfinite(tab_gall['redshift_qsoc'])
quants['N_gall_wqsoc'] = f"{np.sum(i_gall_wqsoc):,}"
print(quants['N_gall_wqsoc'])

6,375,063


In [92]:
quants['N_gpurer'] = f"{len(tab_gpurer):,}"
print(quants['N_gpurer'])

1,942,825


In [93]:
i_gpurer_wqsoc = np.isfinite(tab_gpurer['redshift_qsoc'])
quants['N_gpurer_wqsoc'] = f"{np.sum(i_gpurer_wqsoc):,}"
print(quants['N_gpurer_wqsoc'])

1,729,625


In [169]:
quants['N_gsup'] = f"{len(tab_gsup):,}"
print(quants['N_gsup'])

1,518,782


In [168]:
quants['N_gclean'] = f"{len(tab_gclean):,}"
print(quants['N_gclean'])

1,442,077


In [172]:
quants['p_cut_gsup_gclean'] = f"{100*(len(tab_gsup) - len(tab_gclean))/len(tab_gsup):.0f}"
print(quants['p_cut_gsup_gclean'])

5


In [137]:
quants['N_gcatlo'] = f"{len(tab_gcatlo):,}"
print(quants['N_gcatlo'])

767,866


In [138]:
quants['N_gcathi'] = f"{len(tab_gcathi):,}"
print(quants['N_gcathi'])

1,318,566


In [161]:
i_squasars_unwise = np.isfinite(tab_squasars['mag_w1_vg']) & ~tab_squasars.mask['mag_w1_vg'] & \
                    np.isfinite(tab_squasars['mag_w2_vg']) & ~tab_squasars.mask['mag_w2_vg']
quants['N_squasars_unwise'] = f"{np.sum(i_squasars_unwise):,}"
print(quants['N_squasars_unwise'])

343,074


In [167]:
i_sq_sup = tab_labeled_sup['class']=='q'
quants['N_squasars_sup'] = f"{np.sum(i_sq_sup):,}"
print(quants['N_squasars_sup'])

246,122


## Redshift info

### general

In [94]:
i_zfinite = np.isfinite(tab_gall['redshift_qsoc'])
quants['z_med_gall'] = f"{np.median(tab_gall['redshift_qsoc'][i_zfinite]):.2f}"
print(quants['z_med_gall'])

1.67


In [177]:
quants['z_med_gcatlo'] = f"{np.median(tab_gcatlo['redshift_spz']):.2f}"
print(quants['z_med_gcatlo'])

1.45


In [181]:
zintermediate = 2.5
quants['zintermediate'] = zintermediate
i_above_zintermediate_gcatlo = tab_gcatlo['redshift_spz'] > zintermediate
quants['N_above_zintermediate_gcatlo'] = np.sum(i_above_zintermediate_gcatlo)
quants['p_above_zintermediate_gcatlo'] = np.sum(i_above_zintermediate_gcatlo)/len(i_above_zintermediate_gcatlo)
print(quants['N_above_zintermediate_gcatlo'], quants['p_above_zintermediate_gcatlo'])

79441 0.1034568531488567


### dz checks vs sdss

In [95]:
rand_ints_labeled = tab_spz_labeled['rand_ints']
i_train, i_valid, i_test = utils.split_train_val_test(rand_ints_labeled,
                                 frac_train=0.7, frac_val=0.15, frac_test=0.15)

In [96]:
z_sdss_test = tab_spz_labeled['z_sdss'][i_test]
z_spzraw_test = tab_spz_labeled['redshift_spz_raw'][i_test]
z_spz_test = tab_spz_labeled['redshift_spz'][i_test]
z_gaia_test = tab_spz_labeled['redshift_qsoc'][i_test]

In [97]:
dz_spzraw_test = (z_spzraw_test - z_sdss_test)/(1+z_sdss_test)
dz_spz_test = (z_spz_test - z_sdss_test)/(1+z_sdss_test)
dz_gaia_test = (z_gaia_test - z_sdss_test)/(1+z_sdss_test)

In [183]:
dz_threshs = {'dzlo': 0.01, 'dzmid': 0.1, 'dzhi': 0.2}
G_maxs = {'Gbright': 19.0, 'Glo': 20.0, 'Ghi': 20.5}
dz_arrs = {'zspz': dz_spz_test, 'zgaia': dz_gaia_test}

In [184]:
for G_name, G_max in G_maxs.items():
    quants[G_name] = G_max
    for dz_thresh_name, dz_thresh in dz_threshs.items():
        quants[dz_thresh_name] = dz_thresh
        for z_name, dz_arr in dz_arrs.items():
        
            i_G = tab_spz_labeled['phot_g_mean_mag'][i_test] < G_max
            i_outliers = np.abs(dz_arr[i_G]) > dz_thresh
            key = f'p_outliers_{z_name}_{dz_thresh_name}_{G_name}'
            val = f"{100*np.sum(i_outliers)/len(i_outliers):.0f}"
            print(key, val)
            quants[key] = val
            
            i_acc = np.abs(dz_arr[i_G]) <= dz_thresh
            key = f'p_acc_{z_name}_{dz_thresh_name}_{G_name}'
            val = f"{100*np.sum(i_acc)/len(i_acc):.0f}"
            print(key, val)
            quants[key] = val

p_outliers_zspz_dzlo_Gbright 14
p_acc_zspz_dzlo_Gbright 86
p_outliers_zgaia_dzlo_Gbright 9
p_acc_zgaia_dzlo_Gbright 91
p_outliers_zspz_dzmid_Gbright 5
p_acc_zspz_dzmid_Gbright 95
p_outliers_zgaia_dzmid_Gbright 7
p_acc_zgaia_dzmid_Gbright 93
p_outliers_zspz_dzhi_Gbright 3
p_acc_zspz_dzhi_Gbright 97
p_outliers_zgaia_dzhi_Gbright 6
p_acc_zgaia_dzhi_Gbright 94
p_outliers_zspz_dzlo_Glo 26
p_acc_zspz_dzlo_Glo 74
p_outliers_zgaia_dzlo_Glo 25
p_acc_zgaia_dzlo_Glo 75
p_outliers_zspz_dzmid_Glo 10
p_acc_zspz_dzmid_Glo 90
p_outliers_zgaia_dzmid_Glo 19
p_acc_zgaia_dzmid_Glo 81
p_outliers_zspz_dzhi_Glo 5
p_acc_zspz_dzhi_Glo 95
p_outliers_zgaia_dzhi_Glo 18
p_acc_zgaia_dzhi_Glo 82
p_outliers_zspz_dzlo_Ghi 39
p_acc_zspz_dzlo_Ghi 61
p_outliers_zgaia_dzlo_Ghi 38
p_acc_zgaia_dzlo_Ghi 62
p_outliers_zspz_dzmid_Ghi 17
p_acc_zspz_dzmid_Ghi 83
p_outliers_zgaia_dzmid_Ghi 30
p_acc_zgaia_dzmid_Ghi 70
p_outliers_zspz_dzhi_Ghi 9
p_acc_zspz_dzhi_Ghi 91
p_outliers_zgaia_dzhi_Ghi 28
p_acc_zgaia_dzhi_Ghi 72


Get numbers for all Gaia candidates with SDSS redshifts:

In [107]:
tab_squasars.columns

<TableColumns names=('ra','dec','source_id','ra_sdss','dec_sdss','objid','z_sdss','phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag','phot_bp_n_obs','phot_rp_n_obs','dec_unwise','mag_w1_vg','mag_w2_vg','ra_unwise','unwise_objid','dist_arcsec')>

In [115]:
tab_squasars_lite = tab_squasars.copy()
tab_squasars_lite.keep_columns(['source_id', 'z_sdss'])
tab_gall_lite = tab_gall.copy()
tab_gall_lite.keep_columns(['source_id', 'redshift_qsoc', 'phot_g_mean_mag'])

In [116]:
tab_gall_xsquasars = join(tab_gall_lite, tab_squasars_lite, keys='source_id', join_type='inner')
print(len(tab_gall_xsquasars))

326067


In [118]:
tab_gall_xsquasars = tab_gall_xsquasars[np.isfinite(tab_gall_xsquasars['redshift_qsoc'])]
print(len(tab_gall_xsquasars))

320149


In [119]:
z_sdss_gall = tab_gall_xsquasars['z_sdss']
z_gaia_gall = tab_gall_xsquasars['redshift_qsoc']

dz_gaia_gall = (z_gaia_gall - z_sdss_gall)/(1+z_sdss_gall)

In [127]:
z_name = 'zgaia'
for G_name, G_max in G_maxs.items():
    for dz_thresh_name, dz_thresh in dz_threshs.items():
        
        i_G = tab_gall_xsquasars['phot_g_mean_mag'] < G_max
        i_outliers = np.abs(dz_gaia_gall[i_G]) > dz_thresh
        key = f'p_outliers_gall_{z_name}_{dz_thresh_name}_{G_name}'
        val = f"{100*np.sum(i_outliers)/len(i_outliers):.0f}"
        print(key, val)
        quants[key] = val

p_outliers_gall_zgaia_dzlo_Glo 25
p_outliers_gall_zgaia_dzmid_Glo 19
p_outliers_gall_zgaia_dzhi_Glo 18
p_outliers_gall_zgaia_dzlo_Ghi 38
p_outliers_gall_zgaia_dzmid_Ghi 30
p_outliers_gall_zgaia_dzhi_Ghi 28


Ends up being the same as our cleaned sample lol oop (rounded to a percent)

In [142]:
G_name = 'Glo'
for dz_thresh_name, dz_thresh in dz_threshs.items():
    fac = float(quants[f'p_outliers_gall_zgaia_{dz_thresh_name}_{G_name}'])/ \
          float(quants[f'p_outliers_zspz_{dz_thresh_name}_{G_name}'])
    key = f'factor_reduction_outliers_{dz_thresh_name}_{G_name}'
    fac_rounded = round(fac*2)/2
    val = rf'${{\sim}}{fac_rounded:g}\times$'
    print(fac)
    print(key,':', val)
    quants[key] = val

0.9615384615384616
factor_reduction_outliers_dzlo_Glo : ${\sim}1\times$
1.9
factor_reduction_outliers_dzmid_Glo : ${\sim}2\times$
3.6
factor_reduction_outliers_dzhi_Glo : ${\sim}3.5\times$


### Decontamination

In [154]:
i_contam_labeled = (tab_labeled_sup['class']=='s') | (tab_labeled_sup['class']=='g')
print(np.sum(i_contam_labeled))

i_contam_gclean = np.isin(tab_labeled_sup['source_id'][i_contam_labeled], tab_gclean['source_id'])
print(np.sum(i_contam_gclean))

3592
1138


In [155]:
key = 'factor_reduction_contaminants'
fac = np.sum(i_contam_labeled)/np.sum(i_contam_gclean)
fac_rounded = round(fac*2)/2
val = rf'${{\sim}}{fac_rounded:g}\times$'
print(fac)
print(key, val)
quants[key] = val

3.156414762741652
factor_reduction_contaminants ${\sim}3\times$


In [272]:
i_q_labeled = tab_labeled_sup['class']=='q'
print(np.sum(i_q_labeled))

i_q_gclean = np.isin(tab_labeled_sup['source_id'][i_q_labeled], tab_gclean['source_id'])
print(np.sum(i_q_gclean))

246122
245450


In [275]:
key = 'p_sq_excluded'
p = np.sum(i_q_labeled)/np.sum(i_q_gclean)-1
print(p)
val = f'{p*100:.1f}'
print(key, val)
quants[key] = val

0.00273782847830506
p_sq_excluded 0.3


### Area and volume

In [254]:
def ndens_of_z(z_arr, z_bins, cosmo):
    
    v_at_z_bins = np.empty(len(z_bins), dtype=u.Quantity)
    for i in range(len(z_bins)):
        v = cosmo.comoving_volume(z_bins[i])
        v_at_z_bins[i] = v.to(u.Gpc**3)
    v_shells = v_at_z_bins[1:] - v_at_z_bins[:-1] 
    
    ndens = []
    for i in range(len(z_bins)-1):
        N_inbin = np.sum((z_arr >= z_bins[i]) & (z_arr < z_bins[i+1]))
        ndens_inbin = N_inbin/v_shells[i]
        ndens.append(ndens_inbin)
    return ndens

In [255]:
def volume_effective_Gpcperh(z_arr, z_bins, z_min, z_max, cosmo, area, P0):
    
    area_rad = area.to(u.rad**2)
    ndens_at_z_bins = ndens_of_z(z_arr, z_bins, cosmo)
    
    v_at_z_bins = np.empty(len(z_bins), dtype=u.Quantity)
    R_at_z_bins = np.empty(len(z_bins), dtype=u.Quantity)
    for i in range(len(z_bins)):
        v = cosmo.comoving_volume(z_bins[i])
        v_at_z_bins[i] = v.to(u.Gpc**3)
        R_at_z_bins[i] = cosmo.comoving_distance(z_bins[i]).to(u.Gpc)
    v_shells = v_at_z_bins[1:] - v_at_z_bins[:-1] 
    
    # discrete integral, just summing
    def integrand(ndens, R, dRdz, dz):
        return area_rad.value * R**2 * dRdz * dz

    integral = 0
    for i in range(len(z_bins)-1):
        z_avg = 0.5*(z_bins[i] + z_bins[i+1])
        
        R = cosmo.comoving_distance(z_avg).to(u.Gpc)
        dR = R_at_z_bins[i+1] - R_at_z_bins[i]
        dz = z_bins[i+1] - z_bins[i]
        dRdz = dR/dz
        integral += integrand(ndens_at_z_bins[i], R, dRdz, dz)

    v_Gpc = integral.to(u.Gpc**3)
    v_Gpcperh = v_Gpc * cosmo.h**3
#     print(f"Sky area is {area:.3f} which is {area/area_allsky:.3f} of the sky")
#     print(f"Effective volume is {v_Gpc:.3f} = {v_Gpcperh.value:.3f} (Gpc/h)^3")
    return v_Gpcperh.value

In [261]:
cosmo = Planck18
area_allsky = 41252.96125*(u.deg**2)

TODO: estimating p0 now, what should be doing?

In [245]:
fn_dustmap = f'../data/maps/map_dust_NSIDE{NSIDE}.npy'
map_dust = maps.get_dust_map(NSIDE=NSIDE, R=3.1, fn_map=fn_dustmap)

Dustmap already exists, loading from ../data/maps/map_dust_NSIDE64.npy


Gaia:

In [264]:
Av_hi = 0.5
npix_below_Avhi = np.sum(map_dust < Av_hi)
area_below_Avhi = area_per_pixel*u.deg**2 * npix_below_Avhi
quants['Avhi'] = Av_hi
area_below_Avhi_fmt = rf'{area_below_Avhi.value:.2f} deg$^2$'
quants['area_below_Avhi'] = area_below_Avhi_fmt
print(quants['area_below_Avhi'])

29154.54 deg$^2$


In [268]:
z_arr = tab_gcathi['redshift_spz']
z_min, z_max = np.min(z_arr), np.max(z_arr)
z_bins = np.arange(0, z_max+0.1, 0.1)
P0 = 10**4 * u.Mpc**3 * cosmo.h**3 
vol_gcathi = volume_effective_Gpcperh(z_arr, z_bins, z_min, z_max, cosmo, area_below_Avhi, P0)
vol_gcathi_fmt = rf'{vol_gcathi:.2f} $(h^{{-1}}\,Gpc)^3$'
quants['vol_gcathi_below_Avhi'] = vol_gcathi_fmt
print(quants['vol_gcathi_below_Avhi'])

Sky area is 29154.543 deg2 which is 0.707 of the sky
Effective volume is 1393.590 Gpc3 = 431.649 (Gpc/h)^3
431.6493335228083
431.65 $(h^{-1}\,Gpc)^3$


TODO: effective number density

SDSS:

In [269]:
# area via: # https://academic.oup.com/mnras/article/514/3/3396/6566355?login=true
area_sdss = 4808*(u.deg**2)

In [270]:
z_arr = tab_sdss_all['z_sdss']
z_min, z_max = np.min(z_arr), np.max(z_arr)
z_bins = np.arange(0, z_max+0.1, 0.1)
P0 = 10**4 * u.Mpc**3 * cosmo.h**3 
vol_sdss = volume_effective_Gpcperh(z_arr, z_bins, z_min, z_max, cosmo, area_sdss, P0)
vol_sdss_fmt = rf'{vol_sdss:.2f} $(h^{{-1}}\,Gpc)^3$'
quants['vol_sdss'] = vol_sdss_fmt
print(quants['vol_sdss'])

NameError: name 'tab_sdss_all' is not defined

TODO: get sdss data and run!

Gaia vs SDSS

In [ ]:
fac_area = area_below_Avhi/area_sdss
fac_area_rounded = round(fac_area*2)/2
val = rf'${{\sim}}{fac_area_rounded:g}\times$'
quants['factor_area_gcathi_sdss'] = val
print(quants['factor_area_gcathi_sdss'])

In [ ]:
fac_vol = vol_gcathi/vol_sdss
fac_vol_rounded = round(fac_vol*2)/2
val = rf'${{\sim}}{fac_vol_rounded:g}\times$'
quants['factor_vol_gcathi_sdss'] = val
print(quants['factor_vol_gcathi_sdss'])

# Write dict to file

In [182]:
with open(fn_quants, 'w') as f:
    for key, val in quants.items():
        print(key, val)
        if save:
            f.write(f'{key} = {val}\n')
if save:
    print()
    print(f"Saved to {fn_quants}!")

N_gall 6,649,162
N_gall_wqsoc 6,375,063
N_gpurer 1,942,825
N_gpurer_wqsoc 1,729,625
z_med_gall 1.67
Glo 20.0
dzlo 0.01
p_outliers_dzlo_gall_Glo 74
dzmid 0.1
p_outliers_dzmid_gall_Glo 90
dzhi 0.2
p_outliers_dzhi_gall_Glo 95
Ghi 20.5
p_outliers_dzlo_gall_Ghi 61
p_outliers_dzmid_gall_Ghi 83
p_outliers_dzhi_gall_Ghi 91
p_outliers_zspz_dzhi_Glo 5
p_outliers_zgaia_dzhi_Glo 18
p_outliers_zspz_dzhi_Ghi 9
p_outliers_zgaia_dzhi_Ghi 28
p_outliers_zspz_dzlo_Glo 26
p_outliers_zgaia_dzlo_Glo 25
p_outliers_zspz_dzmid_Glo 10
p_outliers_zgaia_dzmid_Glo 19
p_outliers_zspz_dzlo_Ghi 39
p_outliers_zgaia_dzlo_Ghi 38
p_outliers_zspz_dzmid_Ghi 17
p_outliers_zgaia_dzmid_Ghi 30
p_outliers_gall_zgaia_dzlo_Glo 25
p_outliers_gall_zgaia_dzmid_Glo 19
p_outliers_gall_zgaia_dzhi_Glo 18
p_outliers_gall_zgaia_dzlo_Ghi 38
p_outliers_gall_zgaia_dzmid_Ghi 30
p_outliers_gall_zgaia_dzhi_Ghi 28
factor_reduction_outliers_dzlo_Glo ${\sim}1\times$
factor_reduction_outliers_dzmid_Glo ${\sim}2\times$
factor_reduction_outliers_dzhi

## Table showing catalog

In [185]:
tab_gcatlo.columns

<TableColumns names=('source_id','ra','dec','l','b','phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag','phot_g_n_obs','phot_bp_n_obs','phot_rp_n_obs','redshift_qsoc','redshift_qsoc_lower','redshift_qsoc_upper','zscore_qsoc','flags_qsoc','pmra','pmra_error','pmdec','pmdec_error','parallax','parallax_error','ebv','A_v','pm','rand_ints','mag_w1_vg','mag_w2_vg','unwise_objid','g_rp','bp_g','bp_rp','g_w1','w1_w2','redshift_spz_raw','redshift_spz_err','redshift_spz')>

In [239]:
columns_show = ['source_id', 'ra', 'dec', 'redshift_spz', 'redshift_spz_err', 'unwise_objid']
#columns_show = ['source_id', 'ra', 'dec']
fmt_dict = {'source_id': 'd',
            'ra': '.7f',
            'dec': '.7f', 
            'redshift_spz': '.4f',
            'redshift_spz_err': '.4f',
            'unwise_objid': ''
            }

In [240]:
n_show = 5
rand_ints_show = np.arange(5)
table_contents = ''

column_labels = [f'\\texttt{{{c}}}' for c in columns_show]
table_contents += ' & '.join(column_labels) + '\\\\ \n'
for rand_int in rand_ints_show:
    idx = np.where(tab_gcatlo['rand_ints']==rand_int)[0][0]
    vals = [tab_gcatlo[c][idx] for c in columns_show]
    print(vals)
    #table_contents += ' & '.join(f'{vals:{fmt}}') + '\\\\ \n'
    vals_fmt = [f'{vals[i]:{fmt_dict[columns_show[i]]}}' for i in range(len(columns_show))]
    table_contents += ' & '.join(vals_fmt) + ' \\\\ \n'
    
print(table_contents)

[6879983591771489024, 305.83261978244406, -12.188166605726837, 2.1306028366088867, 0.6760891064071656, '3059m122o0016112']
[4794863040707918080, 88.98124338469555, -48.262755725696955, 0.819059431552887, 0.055243764801025386, '0899m485o0023638']
[6539481853018479872, 346.84816729684184, -46.337839769077696, 2.166107416152954, 0.0602146595382691, '3469m470o0026036']
[4226860693919442688, 312.42045020208616, -1.172894041409968, 1.5396114587783813, 0.06936078229904175, '3131m016o0024412']
[3598429879556184064, 177.417328564875, -6.631830427786866, 2.560123920440674, 0.3770664857101441, '1777m061o0002874']
\texttt{source_id} & \texttt{ra} & \texttt{dec} & \texttt{redshift_spz} & \texttt{redshift_spz_err} & \texttt{unwise_objid}\\ 
6879983591771489024 & 305.8326198 & -12.1881666 & 2.1306 & 0.6761 & 3059m122o0016112 \\ 
4794863040707918080 & 88.9812434 & -48.2627557 & 0.8191 & 0.0552 & 0899m485o0023638 \\ 
6539481853018479872 & 346.8481673 & -46.3378398 & 2.1661 & 0.0602 & 3469m470o0026036 \

In [241]:
fn_table = '../data/catalog_table.txt'
with open(fn_table, 'w') as f:
    if save:
        f.write(table_contents)
if save:
    print(f"Saved to {fn_table}!")

Saved to ../data/catalog_table.txt!


TODO: write checks for randints for every table

TODO: fix header for final catalog